<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E5%88%A9%E7%94%A8qwen%E6%A8%A1%E5%9E%8B%E8%BF%9B%E8%A1%8C%E5%AF%B9%E8%AF%9D%E6%8A%BD%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 找一本小说
- [x] 切章节，600个token一组形成list of dict，标记章节头
- [ ] 初始化2个或者4个qwen实例
- [ ] 用async函数进行批量调用
- [ ] 算一下peritem的时间

先从drive上找到小说

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/Wuxia/input/

1819部精校小说.zip  classic_50.zip  parts_1.zip  parts_3.zip  parts_5.zip  parts_7.zip	parts_9.zip
all_direct	    parts_0.zip     parts_2.zip  parts_4.zip  parts_6.zip  parts_8.zip


In [6]:
part_id = 11

这个zip在

https://drive.google.com/file/d/101g9BfaKLNv10P1C1wuePTdbR6rwKRpb/view?usp=sharing

In [5]:
# part_id = 10

# input_file = f"/content/drive/MyDrive/Wuxia/input/parts_" + str(part_id) + ".zip"

input_file = "/content/drive/MyDrive/Wuxia/input/classic_50.zip"

import os
import zipfile
import shutil

output_dir = "/content/input"

# 解压input_file到output_dir
with zipfile.ZipFile(input_file,"r") as zip_ref:
    zip_ref.extractall(output_dir)

In [9]:


import os
import chardet

output_folder = "/content/input/output2"
# output_folder = f"/content/output/parts_{part_id}"


file_names = []

# 遍历所有txt文件
for root, dirs, files in os.walk(output_folder):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)

            with open(file_path, 'rb') as f:
                content = f.read(1000)
            result = chardet.detect(content)

            if result['confidence'] < 0.9:
                print(f"Warning: {file_path} encoding confidence {result['confidence']:.0%} lower than 90%")
                continue

            file_names.append({
                "file_name": file_path,
                "encoding": result['encoding'],
                "confidence": result['confidence']
            })

In [11]:
for file_name in file_names[:5]:
    print(file_name)

{'file_name': '/content/input/output2/《娘要嫁人》（精校全本）作者：严歌苓.txt', 'encoding': 'GB2312', 'confidence': 0.99}
{'file_name': '/content/input/output2/《鬼吹灯Ⅰ》（精校全本）作者：天下霸唱.txt', 'encoding': 'UTF-16', 'confidence': 1.0}
{'file_name': '/content/input/output2/《金枝玉叶(九重凤阙)》（校对版全本+番外）作者：灯火阑珊.txt', 'encoding': 'UTF-16', 'confidence': 1.0}
{'file_name': '/content/input/output2/《悟空传》（精校全本）作者：今何在.txt', 'encoding': 'GB2312', 'confidence': 0.99}
{'file_name': '/content/input/output2/《金陵十三钗》（校对全本）作者：严歌苓.txt', 'encoding': 'GB2312', 'confidence': 0.99}


In [14]:
import re

def get_file_name(fname):
    last_split = fname.split('/')[-1]

    pattern = r'《(.+)》'
    m = re.match(pattern, last_split)
    if m:
        # print("match")
        return m.group(1) + '.txt'
    else:
        return last_split

fname = "/content/output/output2/《凡人修仙传》(精校全本)作者:忘语.txt"

print(get_file_name(fname)[:-4])

凡人修仙传


In [15]:
def predict_chapater(line):
    line = line.strip()
    line = line.strip('-=*')
    if len(line) > 100:
        return -1
    if len(line) == 0:
        return -1
    is_short = len(line) < 10

    head_5_char = line[:min(5,len(line))]

    small_line = line.lower()
    line = line[:20]
    has_di = line.find('第', 0 ) >= 0
    di_in_head = head_5_char.find('第', 0) >= 0

    has_zhang = False
    zhang_in_head = False

    zhang_word = ['章', '集', '卷','回','期','节']



    for w in zhang_word:
        if head_5_char.find(w, 0) >= 0:
            zhang_in_head = True
            break

    for w in zhang_word:
        if line.find(w, 0) >= 0:
            has_zhang = True
            break

    has_digital = False
    digital_word = ['0','1','2','3','4','5','6','7','8','9',\
                    '一','二','三','四','五','六','七','八','九']

    only_digital = False
    digital_count = 0
    for ch in line:
        if ch in digital_word:
            digital_count += 1
    if digital_count == len(line):
        only_digital = True

    digital_in_head = False

    for w in digital_word:
        if head_5_char.find(w, 0) >= 0:
            digital_in_head = True
            break
    for w in digital_word:
        if line.find(w, 0) >= 0:
            has_digital = True
            break
    if has_di and has_zhang and has_digital and (di_in_head or digital_in_head or zhang_in_head):
        return 3
    if di_in_head and digital_in_head and is_short:
        return 2
    if zhang_in_head and digital_in_head and is_short:
        return 2
    # if only_digital and is_short:
    #     return 2
    return -1


In [19]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("silk-road/Haruhi-Dialogue-Speaker-Extract-csv", trust_remote_code=True)


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/64.0 [00:00<?, ?B/s]

In [21]:
print(len(tokenizer.tokenize("我是一只猫")))

4


In [25]:
#@title  定义divide函数，用来切分超长文本
def divide_str(s, sep=['\n', '.', '。']):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, mid_len // 2, len(s)-mid_len // 2)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos -
                                                        mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]


def strong_divide(s):
    left, right = divide_str(s)

    if right != '':
        return left, right

    whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']
    left, right = divide_str(s, sep=whole_sep)

    if right != '':
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]

In [22]:
import os

save_folder = "/content/output_in_chunk"

TOKEN_PER_TRUNK = 600

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

import codecs

from tqdm import tqdm
# for file_name_data in tqdm(file_names):
for file_name_data in file_names:
    file_name = file_name_data['file_name']
    if file_name.find("《斗罗大陆》",0)<0:
        # 先找一本进行调试
        continue

    book_name = get_file_name(file_name)
    if len(book_name) > 4:
        book_name = book_name[:-4]
    fencoding = file_name_data['encoding']
    f = codecs.open(file_name, 'r', encoding=fencoding, errors = 'ignore')
    raw_text = f.read()
    lines = raw_text.splitlines()
    f.close()

    head_count = 0

    is_heads = [predict_chapater(line) > 0 for line in lines]

    token_per_line = []

    for line in tqdm(lines):
        if line.strip() == '':
            token_per_line.append(0)
            continue
        n_token = len(tokenizer.tokenize(line.strip()))
        token_per_line.append(n_token)

    # 这里还要处理 如果单个line超过TOKEN_PER_TRUNK, 需要调用strong_divide进行分裂
    # 同时第一个line保留原来的is_head, 其他的都是false


    for line, is_head in zip(lines, is_heads):
        if is_head:
            head_count += 1



    print(file_name_data)
    print(head_count)
    break

100%|██████████| 40233/40233 [00:12<00:00, 3213.49it/s]

{'file_name': '/content/input/output2/《斗罗大陆》（精校全本）作者：唐家三少.txt', 'encoding': 'GB2312', 'confidence': 0.99}
344


In [24]:
print(sum(token_per_line)/600)

3258.015


In [34]:
chunks = []

current_n = 0
current_chunk = ""
current_chunk_is_head = True
last_chunk_is_head = False

for line, is_head, n_token in zip(lines, is_heads, token_per_line):
    # 如果is_head且之前不是head
    # 如果is_head且之前的current_n > TOKEN_PER_TRUNK // 2 需要结算
    # 如果n_token + current_n > TOKEN_PER_TRUNK 需要结算
    count_flag = False

    if is_head and not last_chunk_is_head:
        count_flag = True
    elif is_head and last_chunk_is_head and current_n > TOKEN_PER_TRUNK // 2:
        count_flag = True
    elif n_token + current_n > TOKEN_PER_TRUNK:
        count_flag = True

    if count_flag and current_chunk.strip() != "":
        chunk_data = {
            "text": current_chunk,
            "is_head": current_chunk_is_head
        }
        chunks.append(chunk_data)
        last_chunk_is_head = current_chunk_is_head
        current_chunk_is_head = is_head
        current_chunk = ""
        current_n = 0

    current_chunk += line + "\n"
    current_n += n_token

if current_chunk.strip() != "":
    chunk_data = {
        "text": current_chunk,
        "is_head": current_chunk_is_head
    }
    chunks.append(chunk_data)

In [36]:
print(sum([ chunk["is_head"] == True for chunk in chunks ]))
print(len(chunks))

342
3616


3618
